# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/cities.csv')

weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
lat_lon = weather_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]
lat_lon.head()

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42


In [4]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.Temp < 75) & (weather_df.Temp > 60)
crit_windSpeed = weather_df.Wind_Speed < 15
crit_cloudiness = weather_df.Cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed
0,6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
1,37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
2,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
3,84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
4,114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
5,141,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24
6,168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
7,192,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93
8,214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
9,219,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed,Hotel Name
0,6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,
1,37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
2,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
3,84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,
4,114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,


In [10]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       26 non-null     int64  
 1   City_ID     26 non-null     int64  
 2   City        26 non-null     object 
 3   Cloudiness  26 non-null     int64  
 4   Country     26 non-null     object 
 5   Date        26 non-null     int64  
 6   Humidity    26 non-null     int64  
 7   Lat         26 non-null     float64
 8   Lng         26 non-null     float64
 9   Temp        26 non-null     float64
 10  Wind_Speed  26 non-null     float64
 11  Hotel Name  26 non-null     object 
dtypes: float64(4), int64(5), object(3)
memory usage: 2.6+ KB


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

In [9]:
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…